# Outlook

In [a previous notebook](https://colab.research.google.com/drive/1_yp-JKkxh_P8Yhctulqm0IrLbE41oK1p?usp=sharing), we have seen the BBRL data collection approach and illustrated it with a simple pair of random agents. In this colab we explain how to create RL agents based on neural networks using pytorch and how to create agents representing gym environments using the GymAgent classes from BBRL.

## Installation and imports

The BBRL library is [here](https://github.com/osigaud/bbrl).

Note that we install the `my_gym` library to make sure to import the gym version 0.21.0. The interface of the later versions has been modified and maybe incompatible with previous code.

In [ ]:
import functools
import time

!pip install omegaconf
import omegaconf

import gym
!pip install git+https://github.com/osigaud/my_gym.git

try:
  import bbrl
except ImportError:
  from IPython.display import clear_output 
  !pip install git+https://github.com/osigaud/bbrl.git
  clear_output()
  import bbrl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/osigaud/my_gym.git to /tmp/pip-req-build-3m0u1fzr
  Running command git clone -q https://github.com/osigaud/my_gym.git /tmp/pip-req-build-3m0u1fzr
  Cloning https://github.com/osigaud/SimpleMazeMDP.git to /tmp/pip-install-phz280x3/mazemdp_8a7911bc63c443d1ad833e0c43ad99e0
  Running command git clone -q https://github.com/osigaud/SimpleMazeMDP.git /tmp/pip-install-phz280x3/mazemdp_8a7911bc63c443d1ad833e0c43ad99e0


### BBRL imports

As in the previous notebook, we import BBRL agents

In [ ]:
from bbrl.workspace import Workspace

from bbrl.agents.agent import Agent

from bbrl import get_class, get_arguments, instantiate_class

# Agents(agent1,agent2,agent3,...) executes the different agents the one after the other
# TemporalAgent(agent) executes an agent (e.g an Agent) over multiple timesteps in the workspace, 
# or until a given condition is reached
from bbrl.agents import Agents, TemporalAgent

# GymAgent (resp. AutoResetGymAgent) are agents able to execute a batch of gym environments
# without (resp. with) auto-resetting. These agents produce multiple variables in the workspace: 
# ’env/env_obs’, ’env/reward’, ’env/timestep’, ’env/done’, ’env/initial_state’, ’env/cumulated_reward’, 
# ... When called at timestep t=0, then the environments are automatically reset. 
# At timestep t>0, these agents will read the ’action’ variable in the workspace at time t − 1
from bbrl.agents.gyma import AutoResetGymAgent, NoAutoResetGymAgent

# Not present in the A2C version...
from bbrl.utils.logger import TFLogger

### Other Imports

In [ ]:
import copy
import time

import torch
import torch.nn as nn
import torch.nn.functional as F

import my_gym

In [ ]:
from omegaconf import OmegaConf

## Creating Neural RL agents

We will build two types of agents:
- First, a stochastic actor that outputs a stochastic discrete action given a state. This agent will be made of two parts, one for generating probabilities over actions, and the other one for choosing an action according to these probabilities.
- Second, a deterministic critic that outputs Q-values for discrete actions given a state. Here we will introduce more general functions to build neural networks from a set of specified layer sizes. 

### A probabilistic actor in two parts

Here we replace the simple ActionAgent of the previous notebook with a combination of two agents: the probabilistic agent, which contains the neural network, and the action agent, which selects an action based on the probabilities resulting from the probabilistic agent.

#### Probabilistic Agent

A ProbAgent is a one hidden layer neural network which takes an observation as input and whose output is a probability given by a final softmax layer.

The first layers are built in the `__init__()` function using the simple `nn.Sequential(...)` model from pytorch. We will do something more sophisticated to deal with an arbitrary number of layers later in another notebook.

Now, let us have a look at the `forward()` function, which is called each time the agent performs a step in the environment.

To get the input observation from the environment we call
`observation = self.get(("env/env_obs", t))`
and that to perform an action in the environment we call
`self.set(("action_probs", t), probs)`. In between, we call `torch.softmax()` to get probabilities from the output layer of the network.

In [ ]:
class ProbAgent(Agent):
    def __init__(self, observation_size, hidden_size, n_actions):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(observation_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions),
        )

    def forward(self, t, **kwargs):
        observation = self.get(("env/env_obs", t))
        scores = self.model(observation)
        probs = torch.softmax(scores, dim=-1)
        self.set(("action_probs", t), probs)

#### Actor Agent

The ActorAgent takes action probabilities as input (coming from the ProbAgent) and outputs an action. In the deterministic case it takes the argmax, in the stochastic case it samples from the Categorical distribution. This agent does not have a neural network, it just takes a decision from the output of the ProbAgent.

In [ ]:
class ActorAgent(Agent):
    def __init__(self):
        super().__init__()

    def forward(self, t, stochastic, **kwargs):
        probs = self.get(("action_probs", t))
        if stochastic:
            action = torch.distributions.Categorical(probs).sample()
        else:
            action = probs.argmax(1)

        self.set(("action", t), action)

Note that instead of having a ProbaAgent then an ActorAgent, we could have built a single agent containing both. We are doing this mainly to illustrate the capabilities of BBRL to combine agents.

Note also that this pair of agents is adequate for environment with discrete actions, but we need something different if the environment takes continuous actions. In that case, instead of a categorical distribution, we would rather use a Gaussian distribution. We will build other versions of these agents later in another notebook.

### A deterministic critic agent

The function below builds a multi-layer perceptron where the size of each layer is given in the `size` list. We also specify the activation function of neurons at each layer and optionally a different activation function for the final layer.

In [ ]:
def build_mlp(sizes, activation, output_activation=nn.Identity()):
    layers = []
    for j in range(len(sizes) - 1):
        act = activation if j < len(sizes) - 2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j + 1]), act]
    return nn.Sequential(*layers)

The `DiscreteQAgent` class implements a critic such the one used in DQN. It has one output neuron per action and its output is the Q-value of these actions given the state. 

Note that as any BBRL agent, it has a forward function that takes a time state as input. This forward function outputs the Q-values at the corresponding time step. Additionally, if the critic is used to choose an action, it also outputs the chosen action at the same time step.

Besides, it is also useful to get the network output (as a Q-value or as an action) given a state rather than a time step. This is what the `predict_action` and `predict_value` functions are used for.

In [ ]:
class DiscreteQAgent(Agent):
    def __init__(self, state_dim, hidden_layers, action_dim):
        super().__init__()
        self.is_q_function = True
        self.model = build_mlp(
            [state_dim] + list(hidden_layers) + [action_dim], activation=nn.ReLU()
        )

    def forward(self, t, choose_action=True, **kwargs):
        obs = self.get(("env/env_obs", t))
        q_values = self.model(obs).squeeze(-1)
        self.set(("q_values", t), q_values)
        if choose_action:
            action = q_values.argmax(1)
            self.set(("action", t), action)

    def predict_action(self, obs, stochastic):
        q_values = self.model(obs).squeeze(-1)
        if stochastic:
            probs = torch.softmax(q_values, dim=-1)
            action = torch.distributions.Categorical(probs).sample()
        else:
            action = q_values.argmax(0)
        return action

    def predict_value(self, obs, action):
        q_values = self.model(obs).squeeze(-1)
        return q_values[0][action]

## Creating the environment agent

Now that we have an RL agent, let us create an environment. In the previous notebook, we were doing something simple building on the Agent class. Now we do something more sophisticated, building on the `GymAgent` class and encapsulating an OpenAI gym environment.

### Using a gym environment

The function below creates the environment. In OpenAI gym, an environment can be known by its name, here the string `env_name`. The environment is generally given a maximum number of steps, which is enforced by the `TimeLimit` wrapper. Therefore, we we do not need to add our own TimeLimit: this may break the episode termination behavior.

In [ ]:
def make_env(env_name):
    return gym.make(env_name)

To call the above function, we will use a reflexive instantiation mechanism and get the parameters of the function from the `params` dictionary, in the `"env":{
      "classname": "__main__.make_env",
      "env_name": "CartPole-v1",
    }` part.

Using this instantiation approach from a function is useful if you define a new env for instance i.e you just change the 'classname' and put the arguments of the constructor directly and everything will work fine. This may be not natural a first sight, but if you start to use it, you will never go back again :) 

The `instantiate_class`, `get_class` and `get_arguments` functions are available in the [`main/bbrl/__init__.py`](https://github.com/osigaud/bbrl/blob/master/bbrl/__init__.py) file. The `get_class` function reads the `classname` in the parameters to create the appropriate type of object, and the `get_arguments` function reads the local paremeters and their values to set them into the corresponding object. 

### Creating the environment agent

Now, let us create the agent representing the environment. We do so using the `NoAutoResetGymAgent` which inherits from the [GymAgent](https://github.com/osigaud/bbrl/blob/master/bbrl/agents/gyma.py#L76) class and is provided by BBRL.
Essential information about this class is given in [this notebook](https://colab.research.google.com/drive/1EX5O03mmWFp9wCL_Gb_-p08JktfiL2l5?usp=sharing).

In the `get_env(cfg)` function below, the `NoAutoResetGymAgent` is created taking as arguments the environment creation function (here the `make_env` function that we defined above) with its parameters, then the number of environments and a seed. This seed serves to initialize the random number generator so that using the same seed will generate the same numbers again.

These parameters are specified into a specific dictionary called `config` which is described below.

In [ ]:
def get_env(cfg):
    env_agent = NoAutoResetGymAgent(
        get_class(cfg.gym_env),
        get_arguments(cfg.gym_env),
        cfg.algorithm.n_envs,
        cfg.algorithm.seed,
    )
    return env_agent

## Running the agents along a single NoAutoReset episode

Before running everything, we need to get the parameters of the algorithm and the environment agent. To do this, we use the `omegaconf` package which builds a dictionary from a configuration string.

In [ ]:
params={
  "algorithm":{
    "seed": 432,
    "n_envs": 1,
    "architecture":{"hidden_size": [32]},
  },
  "gym_env":{
    "classname": "__main__.make_env",
    "env_name": "CartPole-v1",
  },
}

As in [this notebook](https://colab.research.google.com/drive/1_yp-JKkxh_P8Yhctulqm0IrLbE41oK1p?usp=sharing), once we have the RL agent and the environment agent, we bind them together into a TemporalAgent

We have to create the environnement agent first, because we need to know the size of the observation and action spaces to create the action agent which will interact with it.

In [ ]:
config = OmegaConf.create(params)

env_agent = get_env(config)
observation_size, n_actions = env_agent.get_obs_and_actions_sizes()
prob_agent = ProbAgent(observation_size, config.algorithm.architecture.hidden_size, n_actions)
action_agent = ActorAgent()
composed_agent = Agents(env_agent, prob_agent, action_agent)
  
# Get a temporal agent that can be executed in a workspace
t_agent = TemporalAgent(composed_agent)

And finally we execute it in the workspace. Here, we run for 30 steps.

In [ ]:
config = OmegaConf.create(params)

env_agent = get_env(config)
observation_size, n_actions = env_agent.get_obs_and_actions_sizes()
action_agent = DsicreteQAgent(observation_size, config.algorithm.architecture.hidden_size, n_actions)
composed_agent = Agents(env_agent, prob_agent, action_agent)
  
# Get a temporal agent that can be executed in a workspace
t_agent = TemporalAgent(composed_agent)

# We create a workspace
workspace = Workspace()

# The temporal agent will be run for 10 steps on this workspace
t_agent(workspace, t=0, n_steps=30, stochastic=True)

# We retrieve the information as they are stored into the workspace
obs, action, reward, done = workspace["env/env_obs", "action", "env/reward", "env/done"]

# And we print them
print("obs:", obs)
print("action:", action)
print("reward:", reward)
print("done:", done)
# You should see that each variable has been recorded for the number of specified 
# time steps...

obs: tensor([[[-3.0960e-02,  4.1176e-02,  1.7323e-02, -4.8666e-02]],

        [[-3.0137e-02,  2.3605e-01,  1.6350e-02, -3.3583e-01]],

        [[-2.5416e-02,  4.0695e-02,  9.6332e-03, -3.8039e-02]],

        [[-2.4602e-02, -1.5456e-01,  8.8724e-03,  2.5767e-01]],

        [[-2.7693e-02,  4.0430e-02,  1.4026e-02, -3.2204e-02]],

        [[-2.6885e-02,  2.3535e-01,  1.3382e-02, -3.2043e-01]],

        [[-2.2178e-02,  4.3028e-01,  6.9731e-03, -6.0886e-01]],

        [[-1.3572e-02,  6.2530e-01, -5.2041e-03, -8.9934e-01]],

        [[-1.0661e-03,  8.2049e-01, -2.3191e-02, -1.1937e+00]],

        [[ 1.5344e-02,  6.2568e-01, -4.7064e-02, -9.0833e-01]],

        [[ 2.7857e-02,  4.3122e-01, -6.5231e-02, -6.3080e-01]],

        [[ 3.6482e-02,  6.2719e-01, -7.7847e-02, -9.4329e-01]],

        [[ 4.9026e-02,  8.2327e-01, -9.6713e-02, -1.2594e+00]],

        [[ 6.5491e-02,  1.0195e+00, -1.2190e-01, -1.5807e+00]],

        [[ 8.5881e-02,  1.2158e+00, -1.5351e-01, -1.9088e+00]],

        [[ 1.1020e-0

If you run the above interaction loop for enough steps (say 30) and you look closely at the result on the above cell, you will see that after the task is done (that is, the pole falls down in our case), the workspace continues filling data with copies of the last time step.

We explain in details in [this notebook](https://colab.research.google.com/drive/1W9Y-3fa6LsPeR6cBC1vgwBjKfgMwZvP5?usp=sharing)  why this is so, together with several issues about data collection.

### Exercise

Just do the same using a DiscreteQAgent. Print the Q-values stored into the workspace.

In [ ]:
config = OmegaConf.create(params)

env_agent = get_env(config)
observation_size, n_actions = env_agent.get_obs_and_actions_sizes()
action_agent = DiscreteQAgent(observation_size, config.algorithm.architecture.hidden_size, n_actions)
composed_agent = Agents(env_agent, action_agent)
  
# Get a temporal agent that can be executed in a workspace
t_agent = TemporalAgent(composed_agent)
# We create a workspace
workspace = Workspace()

# The temporal agent will be run for 10 steps on this workspace
t_agent(workspace, t=0, n_steps=30, stochastic=True)

# We retrieve the information as they are stored into the workspace
obs, action, reward, done = workspace["env/env_obs", "action", "env/reward", "env/done"]

# And we print them
print("obs:", obs)
print("action:", action)
print("reward:", reward)
print("done:", done)
# You should see that each variable has been recorded for the number of specified 
# time steps...

obs: tensor([[[-0.0310,  0.0412,  0.0173, -0.0487]],

        [[-0.0301, -0.1542,  0.0163,  0.2494]],

        [[-0.0332, -0.3495,  0.0213,  0.5472]],

        [[-0.0402, -0.5450,  0.0323,  0.8466]],

        [[-0.0511, -0.7405,  0.0492,  1.1492]],

        [[-0.0659, -0.9362,  0.0722,  1.4569]],

        [[-0.0846, -1.1322,  0.1013,  1.7713]],

        [[-0.1073, -1.3283,  0.1368,  2.0936]],

        [[-0.1339, -1.5245,  0.1786,  2.4253]],

        [[-0.1643, -1.3313,  0.2271,  2.1924]],

        [[-0.1643, -1.3313,  0.2271,  2.1924]],

        [[-0.1643, -1.3313,  0.2271,  2.1924]],

        [[-0.1643, -1.3313,  0.2271,  2.1924]],

        [[-0.1643, -1.3313,  0.2271,  2.1924]],

        [[-0.1643, -1.3313,  0.2271,  2.1924]],

        [[-0.1643, -1.3313,  0.2271,  2.1924]],

        [[-0.1643, -1.3313,  0.2271,  2.1924]],

        [[-0.1643, -1.3313,  0.2271,  2.1924]],

        [[-0.1643, -1.3313,  0.2271,  2.1924]],

        [[-0.1643, -1.3313,  0.2271,  2.1924]],

        [[-0.16

## What's next?

We are now ready to write a first version of the DQN algorithm, using the `DiscreteQAgent` defined above. We do so in [this notebook](https://colab.research.google.com/drive/1raeuB6uUVUpl-4PLArtiAoGnXj0sGjSV?usp=sharing)


Or we can switch directly to using the AutoResetGymAgent class. We do so in [this notebook](https://colab.research.google.com/drive/1VJUoDGhxKv3mmFjTmLj_JDpappVw29xh?usp=sharing)